The Building Performance Simulation - 6756
Start - 5.30pm
End - pm
8.30pm - 4750
10.19 - 6215
10.41 - 6719

<!-- https://www.tandfonline.com/action/doSearch?AllField=BIM+and+Green+Building&SeriesKey=tbps20 -->


In [5]:
import requests
from bs4 import BeautifulSoup
from pandas.io.html import read_html
import pandas as pd

In [6]:
df = pd.DataFrame()
df.to_csv('blankFile.csv')

columnNames = ['searchTerm1','searchTerm2','url','Title']

df = pd.read_csv("blankFile.csv", names= columnNames , header = None)
df = df.iloc[1:]

for i in range(20):
    section = "section" + str(i)
    sectionData = "sectionData" + str(i)
    df[section] = ''
    df[sectionData] = ''

df


,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19


In [7]:
# searchTerm1 = ['BIM', 'Building Information Modelling']
# searchTerm2 = ['Green Building', 'Green Buildings']
totalSearchCount = 0

searchTerm1 = ['BIM' , 'Building Information Modelling','Building information model','6D BIM', 'Digital construction','Smart construction','Digital engineering']
searchTerm2 = ['Green Building' , 'Green Buildings','Sustainable building','Sustainable buildings','Environmentally friendly design' , 'Sustainability' , 'Sustainable development' , 'Energy','Water' ,'Waste' , 'Thermal comfort','Electricity' ,'Solar','Wind power' , 'Green infrastructure','Life cycle assessment']

counter = 0
programSearchTerms1 = []
programSearchTerms2 = []

for term2 in searchTerm2:
    text2 = ""
    terms2 = term2.split(' ')
    for t2 in terms2:
        text2 = text2 + t2 + "+"

    fileIndex = text2.rfind("+")
    te = text2[:fileIndex]
    programSearchTerms2.append(te)

print(programSearchTerms2)

for term1 in searchTerm1:
    text1 = ""
    terms1 = term1.split(' ')
    for t2 in terms1:
        text1 = text1 + t2 + "+"

    fileIndex = text1.rfind("+")
    te = text1[:fileIndex]
    programSearchTerms1.append(te)

print(programSearchTerms1)


# https://www.tandfonline.com/action/doSearch?AllField=BIM+and+Green+Building&SeriesKey=rcme20
    
baseUrl = 'https://www.tandfonline.com/action/doSearch?AllField='
querySet = []
row = 0
for item1 in programSearchTerms1:
    for item2 in programSearchTerms2:
        
        query = baseUrl + item1 +"+AND+"+ item2+"&SeriesKey=tbps20"
#         print(query)

        response = requests.get(query)
        soup = BeautifulSoup(response.text, 'html.parser')
        allParas = soup.find_all("p")
        dd = allParas[0].text
        miniData = dd.split(' ')
        NumberOfResultsStr = miniData[2]
        indexOfcomma = NumberOfResultsStr.find(',')

        if indexOfcomma != -1:
            NumberOfResultsStr = NumberOfResultsStr.replace(',','')

        number = int(NumberOfResultsStr)
        print(number)
        totalSearchCount = number + totalSearchCount
# print("Total: ",totalSearchCount)        
        
        page = number // 10
        lastPage = number % 10
        if lastPage > 0:
            page = page + 1

        pageSize = page
        print(pageSize)

        cnt = 0
        allLinks = []
        
#         for testing purpose only
#         pageSize =1 
        for currentPage in range(0,pageSize):
            print("In page: ",currentPage)
            
            if currentPage != 0:
                newQuery = query + '&pageSize=10&subjectTitle=&startPage=' + str(currentPage)
                cnt = cnt + 1
            else:
                newQuery = query
            
            print("Searching for query: ",newQuery)
            response = requests.get(newQuery)
            soup = BeautifulSoup(response.text, 'html.parser')
            mydivs = soup.findAll("a", {'class':['ref nowrap']})
            
            for d in mydivs:
                url = 'https://www.tandfonline.com' + d.get('href')
                allLinks.append(url)

        for link in allLinks:
            df.loc[row,'searchTerm1'] = item1
            df.loc[row,'searchTerm2'] = item2
            df.loc[row,'url'] = link
            html_content = requests.get(link).text
            soup = BeautifulSoup(html_content, "lxml")
            
            for i in soup.find('span',{'class':['NLM_article-title' ,'hlFld-title']}):
                print("Title: ",i)
                df.loc[row,'Title'] = i
            
            temp = soup.findAll('h2',{'class':['section-heading-2']} )
            print("Number of sections: ",len(temp))            
            
            count = 0
            for i in range(len(temp)):

                if temp[i].text == temp[i].find_next('p').text:
        
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)                    
                    df.loc[row,sectionData] = extractData
                    count = count + 1
                else:
                    extract = "section" + str(count)
                    extractData = temp[i].find_next('p').text
                    df.loc[row,extract] = temp[i].text
                    sectionData = "sectionData" + str(count)
                    df.loc[row,sectionData] = extractData
                    count = count + 1
            
            
            row = row + 1


['Green+Building', 'Green+Buildings', 'Sustainable+building', 'Sustainable+buildings', 'Environmentally+friendly+design', 'Sustainability', 'Sustainable+development', 'Energy', 'Water', 'Waste', 'Thermal+comfort', 'Electricity', 'Solar', 'Wind+power', 'Green+infrastructure', 'Life+cycle+assessment']
['BIM', 'Building+Information+Modelling', 'Building+information+model', '6D+BIM', 'Digital+construction', 'Smart+construction', 'Digital+engineering']
36
4
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=tbps20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=tbps20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Green+Building&SeriesKey=tbps20&pageSize=10&subjectTitle=&startPage=2
In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField

Title:  An artificial intelligence-based method to efficiently bring CFD to building simulation
Number of sections:  11
Title:  The introduction of building simulation into an architectural faculty: preliminary findings
Number of sections:  6
Title:  Development of building thermal environment emulator to evaluate the performance of the HVAC system operation
Number of sections:  7
Title:  Results variability in accredited building energy performance compliance demonstration software in the UK: an inter-model comparative study
Number of sections:  12
Title:  Thermal simulation outputs: exploring the concept of patterns in design decision-making
Number of sections:  11
Title:  Thermal simulation software outputs: a framework to produce meaningful information for design decision-making
Number of sections:  10
Title:  Architect–BPS consultant collaborations: 
Title:  <i>Harmony or hardship?</i>
Number of sections:  7
Title:  A generative facade design method based on daylighting performanc

Title:  Model implementation and verification of the envelope, HVAC and controller of an office building in Modelica
Number of sections:  11
Title:  Selection criteria for building performance simulation tools: contrasting architects' and engineers' needs
Number of sections:  8
Title:  Modelling diversity in building occupant behaviour: a novel statistical approach
Number of sections:  8
Title:  The introduction of building simulation into an architectural faculty: preliminary findings
Number of sections:  6
Title:  Architect–BPS consultant collaborations: 
Title:  <i>Harmony or hardship?</i>
Number of sections:  7
36
4
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Energy&SeriesKey=tbps20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Energy&SeriesKey=tbps20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+E

Title:  TEASER: an open tool for urban energy modelling of building stocks
Number of sections:  9
Title:  Building simulation visualization for the boardroom: an exploratory study
Number of sections:  10
21
3
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Thermal+comfort&SeriesKey=tbps20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Thermal+comfort&SeriesKey=tbps20&pageSize=10&subjectTitle=&startPage=1
In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=BIM+AND+Thermal+comfort&SeriesKey=tbps20&pageSize=10&subjectTitle=&startPage=2
Title:  A framework to integrate object-oriented physical modelling with building information modelling for building thermal simulation
Number of sections:  9
Title:  Development of building thermal environment emulator to evaluate the performance of the HVAC system operation
Number of sections:  7
Title:  Envelope thermal design

Title:  Thermal simulation outputs: exploring the concept of patterns in design decision-making
Number of sections:  11
Title:  A generative facade design method based on daylighting performance goals
Number of sections:  7
Title:  Thermal simulation software outputs: a framework to produce meaningful information for design decision-making
Number of sections:  10
Title:  Selection criteria for building performance simulation tools: contrasting architects' and engineers' needs
Number of sections:  8
Title:  An instructional design for building energy simulation e-learning: an interdisciplinary approach
Number of sections:  8
Title:  Results variability in accredited building energy performance compliance demonstration software in the UK: an inter-model comparative study
Number of sections:  12
Title:  TEASER: an open tool for urban energy modelling of building stocks
Number of sections:  9
Title:  Architect–BPS consultant collaborations: 
Title:  <i>Harmony or hardship?</i>
Number of se

...

Title:  An instructional design for building energy simulation e-learning: an interdisciplinary approach
Number of sections:  8
Title:  Signature analysis calibration of a school energy model using hourly data
Number of sections:  6
Title:  Simulating use scenarios in hospitals using multi-agent narratives
Number of sections:  9
16
2
In page:  0
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Sustainability&SeriesKey=tbps20
In page:  1
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Sustainability&SeriesKey=tbps20&pageSize=10&subjectTitle=&startPage=1
Title:  Measurements and simulations of thermal comfort: a synagogue in Tel Aviv, Israel
Number of sections:  7
Title:  Bayesian calibration at the urban scale: a case study on a large residential heating demand application in Amsterdam
Number of sections:  8
Title:  Opportunities and pitfalls of using building performance simulation in e

Title:  Bayesian calibration at the urban scale: a case study on a large residential heating demand application in Amsterdam
Number of sections:  8
Title:  Occupants' operation of lighting and shading systems in office buildings
Number of sections:  7
Title:  A model predictive control optimization environment for real-time commercial building application
Number of sections:  7
Title:  Thermal simulation outputs: exploring the concept of patterns in design decision-making
Number of sections:  11
Title:  Measurements and simulations of thermal comfort: a synagogue in Tel Aviv, Israel
Number of sections:  7
Title:  Sensitivity of the predicted convective heat transfer in a cooled room to the computational fluid dynamics simulation approach
Number of sections:  7
Title:  Modelica Buildings library
Number of sections:  10
Title:  Multi-objective optimization of cellular fenestration by an evolutionary algorithm
Number of sections:  10
Title:  The first world championship in cybernetic buil

Title:  The first world championship in cybernetic building optimization
Number of sections:  8
Title:  The importance of integrally simulating the building, HVAC and control systems, and occupants’ impact for energy predictions of buildings including temperature and humidity control: validated case study museum Hermitage Amsterdam
Number of sections:  10
Title:  A review of the application of SimuLink S-functions to multi domain modelling and building simulation
Number of sections:  7
Title:  Bayesian calibration at the urban scale: a case study on a large residential heating demand application in Amsterdam
Number of sections:  8
Title:  Development of a stochastic virtual smart meter data set for a residential building stock – methodology and sample data
Number of sections:  11
Title:  Model-based real-time whole building energy performance monitoring and diagnostics
Number of sections:  6
Title:  Modelica-based modelling and simulation to support research and development in building

In page:  2
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Electricity&SeriesKey=tbps20&pageSize=10&subjectTitle=&startPage=2
In page:  3
Searching for query:  https://www.tandfonline.com/action/doSearch?AllField=Digital+engineering+AND+Electricity&SeriesKey=tbps20&pageSize=10&subjectTitle=&startPage=3
Title:  Signature analysis calibration of a school energy model using hourly data
Number of sections:  6
Title:  Models of domestic occupancy, activities and energy use based on time-use data: deterministic and stochastic approaches with application to various building-related simulations
Number of sections:  8
Title:  Hybrid residential end-use energy and greenhouse gas emissions model – development and verification for Canada
Number of sections:  9
Title:  Development of a stochastic virtual smart meter data set for a residential building stock – methodology and sample data
Number of sections:  11
Title:  Uncertainty and sensitivity d

Title:  A novel method to model trees for building daylighting simulation using hemispherical photography
Number of sections:  8
Title:  Standardization of optimization methodology of daylighting and shading strategy: a case study of an architectural design studio – the German University in Cairo, Egypt
Number of sections:  9
Title:  GlareShade: a visual comfort-based approach to occupant-centric shading systems
Number of sections:  6
Title:  Experimental and numerical investigation of phase change materials with finned encapsulation for energy-efficient buildings
Number of sections:  10
Title:  The first world championship in cybernetic building optimization
Number of sections:  8
Title:  State of the art in lighting simulation for building science: a literature review
Number of sections:  7
Title:  Bayesian calibration at the urban scale: a case study on a large residential heating demand application in Amsterdam
Number of sections:  8
Title:  CyDER – an FMI-based co-simulation platf

In [8]:
df.iloc[:, 0:36]

,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section11,sectionData11,section12,sectionData12,section13,sectionData13,section14,sectionData14,section15,sectionData15
0,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,A framework to integrate object-oriented physi...,Abstract,This paper presents a framework for integratin...,Introduction,A lack of integration between building design ...,Literature review,US DOE (2014) has been providing information o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Semantics for assembling modular components in...,Abstract,This work investigates a scalable building per...,1. Introduction,"Over the last decades, buildings have develope...",2. Related work,Co-simulation of different process models enab...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,An ontology for building monitoring,Abstract,Building data monitoring can provide performan...,1. Introduction,Building performance is commonly specified and...,2. Towards an ontology of building monitoring ...,"It is debatable, if there necessarily exists a...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Modelling diversity in building occupant behav...,Abstract,We propose an advanced modelling framework to ...,1. Introduction,"The physical processes governing heat, air and...",2. A modelling method accounting for occupants...,"Occupants’ interactions with windows, shading ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Building simulation visualization for the boar...,Abstract,"Increasingly, companies strive to deliver proj...",1. Introduction,Building performance simulation (BPS) clearly ...,2. Building performance assessment for the boa...,While there is no research on the exact cognit...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6751,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/full/10.1080/1...,Virtual reality mock-ups for healthcare facili...,Abstract,"In the wake of building information modelling,...",1. Introduction and motivations,Thinking that has emerged in such concepts as ...,2 Related work,Literature documents illustrate developments i...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6752,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/full/10.1080/1...,Building energy model reduction using model-cl...,Abstract,This paper introduces a novel building energy ...,Introduction,The building sector contributes to 30% of the ...,Methodology,The proposed model reduction method is constru...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6753,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/full/10.1080/1...,Development of building thermal environment em...,AbstractFormulae display:?Mathematical formula...,A “performance gap” arises when the actual val...,1. Introduction,"The household and services sectors, both of wh...",2. Review of research related to emulator cons...,An emulator is a simulation model that verifie...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6754,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/full/10.1080/1...,The first world championship in cybernetic bui...,ABSTRACTFormulae display:?Mathematical formula...,ABSTRACT,1. Introduction,Energy consumption related to buildings accoun...,2. References for our competition,"In the past, several attempts have been made t...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df = df.where(pd.notnull(df), None)
df


,searchTerm1,searchTerm2,url,Title,section0,sectionData0,section1,sectionData1,section2,sectionData2,...,section15,sectionData15,section16,sectionData16,section17,sectionData17,section18,sectionData18,section19,sectionData19
0,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,A framework to integrate object-oriented physi...,Abstract,This paper presents a framework for integratin...,Introduction,A lack of integration between building design ...,Literature review,US DOE (2014) has been providing information o...,...,None,None,None,None,None,None,None,None,None,None
1,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Semantics for assembling modular components in...,Abstract,This work investigates a scalable building per...,1. Introduction,"Over the last decades, buildings have develope...",2. Related work,Co-simulation of different process models enab...,...,None,None,None,None,None,None,None,None,None,None
2,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,An ontology for building monitoring,Abstract,Building data monitoring can provide performan...,1. Introduction,Building performance is commonly specified and...,2. Towards an ontology of building monitoring ...,"It is debatable, if there necessarily exists a...",...,None,None,None,None,None,None,None,None,None,None
3,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Modelling diversity in building occupant behav...,Abstract,We propose an advanced modelling framework to ...,1. Introduction,"The physical processes governing heat, air and...",2. A modelling method accounting for occupants...,"Occupants’ interactions with windows, shading ...",...,None,None,None,None,None,None,None,None,None,None
4,BIM,Green+Building,https://www.tandfonline.com/doi/full/10.1080/1...,Building simulation visualization for the boar...,Abstract,"Increasingly, companies strive to deliver proj...",1. Introduction,Building performance simulation (BPS) clearly ...,2. Building performance assessment for the boa...,While there is no research on the exact cognit...,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6751,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/full/10.1080/1...,Virtual reality mock-ups for healthcare facili...,Abstract,"In the wake of building information modelling,...",1. Introduction and motivations,Thinking that has emerged in such concepts as ...,2 Related work,Literature documents illustrate developments i...,...,None,None,None,None,None,None,None,None,None,None
6752,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/full/10.1080/1...,Building energy model reduction using model-cl...,Abstract,This paper introduces a novel building energy ...,Introduction,The building sector contributes to 30% of the ...,Methodology,The proposed model reduction method is constru...,...,None,None,None,None,None,None,None,None,None,None
6753,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/full/10.1080/1...,Development of building thermal environment em...,AbstractFormulae display:?Mathematical formula...,A “performance gap” arises when the actual val...,1. Introduction,"The household and services sectors, both of wh...",2. Review of research related to emulator cons...,An emulator is a simulation model that verifie...,...,None,None,None,None,None,None,None,None,None,None
6754,Digital+engineering,Life+cycle+assessment,https://www.tandfonline.com/doi/full/10.1080/1...,The first world championship in cybernetic bui...,ABSTRACTFormulae display:?Mathematical formula...,ABSTRACT,1. Introduction,Energy consumption related to buildings accoun...,2. References for our competition,"In the past, several attempts have been made t...",...,None,None,None,None,None,None,None,None,None,None


In [10]:
pwd

'/Users/chirag'

In [11]:
cd Desktop/MDSOutput

/Users/chirag/Desktop/MDSoutput


In [12]:
df.to_csv('KG-TBPS6756.csv')